In [13]:
""" workaround for displaying mapboxgl visualizations in Jupyter Notebook.
This is necessary because mapboxgl uses IPython's display module,
which may not be available in some environments.
Added IPython.display as IPython.core.display because of versions mismatch issues.
"""

import sys
from IPython.display import display
sys.modules['IPython.core.display'].display = display

from mapboxgl.viz import ChoroplethViz, LinestringViz
from mapboxgl.utils import create_color_stops
import requests

BASE_URL = "http://localhost:8000"
MAPBOX_TOKEN = "pk.eyJ1Ijoic21pdGluIiwiYSI6ImNtZGdleTlhMDBtb3oyanE1eWltMGpwb2cifQ.IRKtLVZamkmvUCMuD64Ybg"
params = {
"day": "Monday",
"period": "AM Peak"
}
response = requests.get(f"{BASE_URL}/aggregates/", params=params)
geojson_data = response.json()
features = [
    {
        "type": "Feature",
        "geometry": f["geometry"],
        "properties": {
            "average_speed": f["average_speed"],
            "road_name": f["road_name"]
        }
    } for f in geojson_data
]

In [ ]:
""" Version 1: Modify the existing ChoroplethViz to use polygons instead of linestrings.
This version converts WKT LINESTRING to polygons by buffering the linestrings."""
from shapely import wkt
from shapely.geometry import mapping

features_polygon = []
cnt = 0
for feature in features:
    # Parse WKT and buffer to create polygon
    line = wkt.loads(feature['geometry'])
    buffered_polygon = line.buffer(0.0005)  # Small buffer in degrees

    geojson_feature = {
        "type": "Feature",
        "geometry": mapping(buffered_polygon),
        "properties": {
            "average_speed": feature['properties']['average_speed'],
            "road_name": feature['properties']['road_name']
        }
    }
    features_polygon.append(geojson_feature)
    cnt += 1
    if cnt > 1000:  # Limit to first 1000 features for performance
        break

# Now use ChoroplethViz with polygon data
viz = ChoroplethViz(
    {
        "type": "FeatureCollection",
        "features": features_polygon
    },
    access_token=MAPBOX_TOKEN,
    color_property="average_speed",
    color_stops=create_color_stops([10, 20, 30, 40, 50], colors="Reds"),
    center=(-81.6557, 30.3322),
    zoom=11,
    line_width=1.5,
    opacity=0.8,
    legend_style="Average Speed (mph)"
)
viz.show()

In [ ]:
""" Version 0 from theoriginal code: ChoroplethViz with linestrings.
Not working as expected ( does not show lines), but kept for reference."""
viz = ChoroplethViz(
{
"type": "FeatureCollection",
"features": features
},
access_token=MAPBOX_TOKEN,
color_property="average_speed",
color_stops=create_color_stops([10, 20, 30, 40, 50],
colors="Reds"),
center=(-81.6557, 30.3322),
zoom=11,
line_width=1.5,
opacity=0.8,
legend_style="Average Speed (mph)"
)
viz.show()

In [ ]:
import pandas as pd
df = pd.DataFrame([
{
"link_id": f["link_id"],
"avg_speed": f["average_speed"],
"road_name": f["road_name"],
"length": f["length"]
} for f in geojson_data
])
df.sort_values("avg_speed").head(10)

In [ ]:
""" Version 2: Use LinestringViz for linestrings.
Needs to convert WKT LINESTRING to GeoJSON LineString format."""
from mapboxgl import LinestringViz

# First, convert WKT to proper GeoJSON geometry
features_lines = []
for feature in features:  # your array of documents
    # Parse WKT LINESTRING to coordinates
    wkt_geom = feature['geometry']
    coords_str = wkt_geom.replace("LINESTRING(", "").replace(")", "")
    coordinates = [
        [float(coord.split()[0]), float(coord.split()[1])]
        for coord in coords_str.split(",")
    ]

    geojson_feature = {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": coordinates
        },
        "properties": {
            "average_speed": feature['properties']['average_speed'],
            "road_name": feature['properties']['road_name']
        }
    }
    features_lines.append(geojson_feature)

In [ ]:

# Use LinestringViz for line data
viz = LinestringViz(
    {
        "type": "FeatureCollection",
        "features": features_lines
    },
    access_token=MAPBOX_TOKEN,
    color_property="average_speed",
    color_stops=create_color_stops([10, 20, 30, 40, 50], colors="Reds"),
    center=(-81.6557, 30.3322),
    zoom=11
)
viz.show()

In [ ]:
features_lines["road_name"]